# Generate dataset for SARSA algorithm

Two things to set
1. selected_shift: either A or B
2. delta_t: data must available in conversion table

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime

In [2]:
cleaned_trip_df = pd.read_csv('../data/trip_cleaned.csv')

In [3]:
pd.set_option('max_row', 10)

## Convert time to datetime

In [4]:
episode_data = cleaned_trip_df.copy()
episode_data.sort_values(['hack_license', 'pickup_datetime'], inplace=True)
episode_data['pickup_datetime'] = pd.to_datetime(episode_data['pickup_datetime'])
episode_data['dropoff_datetime'] = pd.to_datetime(episode_data['dropoff_datetime'])
episode_data['pickup_time'] = pd.to_datetime(episode_data['pickup_time']).dt.time
episode_data['dropoff_time'] = pd.to_datetime(episode_data['dropoff_time']).dt.time
episode_data['sub_index'] = episode_data.index ##preserve order for the rest of the process

## Get selected shift
* start in the morning
* end in the afternoon

In [5]:
selected_shift = 'A'

In [6]:
def get_shift(time, weekday=True, pickup=True):
    if weekday:
        if pickup:
            if (time > datetime.time(23, 30)) | (time <= datetime.time(11, 30)):
                return 'A'
            else:
                return 'B'
        else:
            if (time > datetime.time(8, 30)) & (time <= datetime.time(20, 30)):
                return 'A'
            else:
                return 'B'            
    else: ## weekend
        if pickup:
            if (time > datetime.time(2, 0)) & (time <= datetime.time(14, 0)):
                return 'A'
            else:
                return 'B'
        else:
            if (time > datetime.time(10, 0)) & (time <= datetime.time(22, 0)):
                return 'A'
            else:
                return 'B'        
        return None

### Label trips with either shift A or B

In [7]:
## determine shift for weekdays
episode_data.loc[(episode_data['pickup_weekday']==1) &
                      (episode_data['first_pickup']==1) , '_PU_shift'] = episode_data['pickup_time']\
                                .apply(lambda x: get_shift(x, weekday=True, pickup=True))
episode_data.loc[(episode_data['dropoff_weekday']==1) &
                      (episode_data['last_dropoff']==1), '_DO_shift'] = episode_data['dropoff_time']\
                                .apply(lambda x: get_shift(x, weekday=True, pickup=False))

## determine shift for weekends
episode_data.loc[(episode_data['pickup_weekday']==0) &
                      (episode_data['first_pickup']==1) , '_PU_shift'] = episode_data['pickup_time']\
                                .apply(lambda x: get_shift(x, weekday=False, pickup=True))
episode_data.loc[(episode_data['dropoff_weekday']==0) &
                      (episode_data['last_dropoff']==1), '_DO_shift'] = episode_data['dropoff_time']\
                                .apply(lambda x: get_shift(x, weekday=False, pickup=False))

## fill shifts for other rows
episode_data['_PU_shift'] = episode_data.groupby('hack_license')['_PU_shift'].ffill()
episode_data['_DO_shift'] = episode_data.groupby('hack_license')['_DO_shift'].bfill()

In [8]:
print('# consistent shift trips: ', episode_data.loc[episode_data['_PU_shift'] == episode_data['_DO_shift']].shape)
print('total trips: ', episode_data.shape)

# consistent shift trips:  (11590795, 36)
total trips:  (13885739, 36)


### Keep only trips with consistent start and end time

In [9]:
## keep only consistent shift
episode_data = episode_data.loc[episode_data['_PU_shift'] == episode_data['_DO_shift']]
episode_data = episode_data.loc[episode_data['_PU_shift'] == selected_shift]
episode_data = episode_data[['sub_index', 'hack_license', 'pickup_datetime', 'dropoff_datetime', 
                            'pickup_taxizone_id', 'dropoff_taxizone_id', 
                            'total_amount', 'first_pickup', 'last_dropoff']].copy()

## Keep only data points of full shift 
A complete shift must have both the first pickup and the last drop off

In [10]:
episode_data['mask_start'] = np.where(episode_data["first_pickup"] == 1, 1, np.nan)
episode_data['mask_start'] = episode_data.groupby('hack_license')['mask_start'].ffill()
episode_data['mask_end'] = np.where(episode_data["last_dropoff"] == 1, 1, np.nan)
episode_data['mask_end'] = episode_data.groupby('hack_license')['mask_end'].bfill()
episode_data['mask'] = np.where((episode_data["mask_start"] == 1) & (episode_data["mask_end"] == 1), 1, np.nan)
episode_data.dropna(subset=['mask'], inplace=True)
episode_data.drop(columns=['mask_start', 'mask_end', 'mask'], inplace=True)

## Assign unique episode id
One shift of a driver is assgined with a unique id

In [11]:
episode_data['episode'] = np.where(episode_data["first_pickup"] == 1, episode_data["first_pickup"].index, np.nan)
episode_data['episode'] = episode_data.groupby('hack_license')['episode'].ffill()

## Convert all time columns to time index
depending on `delta_t`. First round and convert using `interval_index_table`.

In [12]:
interval_index_table = pd.read_csv('../data/interval_index_table_0.csv')
interval_index_table['interval'] = pd.to_datetime(interval_index_table['interval']).dt.time

In [13]:
delta_t = 15
round_by = '{}min'.format(delta_t)
episode_data['pickup_datetime_interval'] = episode_data['pickup_datetime'].dt.round(round_by).dt.time
episode_data['dropoff_datetime_interval'] = episode_data['dropoff_datetime'].dt.round(round_by).dt.time

## convert DO interval to time index
current_conversion = dict(zip(interval_index_table.interval, interval_index_table[f'time_index_{delta_t}m']))
episode_data['pickup_datetime_index'] = [current_conversion[t] for t in episode_data['pickup_datetime_interval']]
episode_data['dropoff_datetime_index'] = [current_conversion[t] for t in episode_data['dropoff_datetime_interval']]

## Collapse Immediate trips into one 

trips/transition must collaspe if current action = 0 and there is immediate transition after the current state

In [14]:
trip_df = episode_data[[ 'episode', 'sub_index', 'pickup_taxizone_id', 'dropoff_taxizone_id', 'pickup_datetime_index',
                        'dropoff_datetime_index', 'total_amount']].copy()

trip_df['pickup_datetime_index_next'] = trip_df.groupby('episode')['pickup_datetime_index'].shift(-1)
trip_df['pickup_taxizone_id_next'] = trip_df.groupby('episode')['pickup_taxizone_id'].shift(-1)

trip_df['dropoff_datetime_index_prev'] = trip_df.groupby('episode')['dropoff_datetime_index'].shift(1)
trip_df['dropoff_taxizone_id_prev'] = trip_df.groupby('episode')['dropoff_taxizone_id'].shift(1)

* Forward and backward fill for rows that should be collapse together
* rewards = sum total amount of those rows
* start location = start location of the first row
* end location = end location of the last row

In [15]:
trip_df['mask_bfill'] = np.where((trip_df['dropoff_taxizone_id']==trip_df['pickup_taxizone_id_next']) &
                                 (trip_df['dropoff_datetime_index']==trip_df['pickup_datetime_index_next']), 1, np.nan)
trip_df['mask_ffill'] = np.where((trip_df['pickup_taxizone_id']==trip_df['dropoff_taxizone_id_prev']) &
                                 (trip_df['pickup_datetime_index']==trip_df['dropoff_datetime_index_prev']), 1, np.nan)

ffill_cols = ['sub_index', 'pickup_taxizone_id', 'pickup_datetime_index']
for col_names in ffill_cols:
    trip_df[col_names] = np.where(trip_df['mask_ffill'] == 1, np.nan, trip_df[col_names])
    trip_df[col_names] = trip_df.groupby('episode')[col_names].ffill()
bfill_cols = ['dropoff_taxizone_id', 'dropoff_datetime_index']
for col_names in bfill_cols:
    trip_df[col_names] = np.where(trip_df['mask_bfill'] == 1, np.nan, trip_df[col_names])
    trip_df[col_names] = trip_df.groupby('episode')[col_names].bfill()

* group data and aggregatedly sum

In [16]:
trip_df = trip_df[['episode', 'sub_index', 'pickup_taxizone_id', 'dropoff_taxizone_id', 'pickup_datetime_index', 
                   'dropoff_datetime_index', 'total_amount']]
trip_df = trip_df.groupby(['episode', 'sub_index', 'pickup_taxizone_id', 'dropoff_taxizone_id', 
                           'pickup_datetime_index', 'dropoff_datetime_index'], as_index=False).agg('sum')
trip_df.sort_values(['episode', 'sub_index'], inplace=True)
trip_df.rename(columns={'pickup_taxizone_id': 'loc',
                        'pickup_datetime_index': 'time',
                        'dropoff_taxizone_id': 'loc_next',
                        'dropoff_datetime_index': 'time_next',
                        'total_amount': 'reward'}, inplace=True)
trip_df['type'] = 0

In [17]:
trip_df.head(10)         

,episode,sub_index,loc,loc_next,time,time_next,reward,type
0,2131.0,2131.0,42.0,166.0,36.0,36.0,9.50,0
1,2131.0,2132.0,238.0,186.0,37.0,38.0,17.65,0
2,2131.0,2133.0,186.0,211.0,39.0,40.0,13.50,0
3,2131.0,2134.0,186.0,234.0,40.0,40.0,8.45,0
4,2131.0,2135.0,90.0,161.0,40.0,41.0,7.50,0
5,2131.0,2136.0,142.0,186.0,41.0,43.0,27.10,0
6,2131.0,2138.0,246.0,211.0,44.0,45.0,15.50,0
7,2131.0,2139.0,114.0,233.0,46.0,48.0,21.00,0
8,2131.0,2141.0,162.0,162.0,48.0,49.0,12.50,0
9,2131.0,2142.0,237.0,236.0,49.0,51.0,14.00,0


## Create reposition dataset

repostion from current DO loc to the next PU location

In [18]:
reposition_df = trip_df.copy()
reposition_df['loc_next_next'] = reposition_df.groupby('episode')['loc'].shift(-1) ## will be next location
reposition_df['time_next_next'] = reposition_df.groupby('episode')['time'].shift(-1) ## will be next time
reposition_df.drop(columns=['loc', 'time'], inplace=True)
reposition_df.rename(columns={'loc_next': 'loc',
                        'time_next': 'time',
                        'loc_next_next': 'loc_next',
                        'time_next_next': 'time_next'}, inplace=True)
reposition_df['reward'] = 0
## 1 = cruise, 2 = repo
reposition_df['type'] = np.where(reposition_df['loc'] == reposition_df['loc_next'], 1, 2)
reposition_df.head(10)

,episode,sub_index,loc,time,reward,type,loc_next,time_next
0,2131.0,2131.0,166.0,36.0,0,2,238.0,37.0
1,2131.0,2132.0,186.0,38.0,0,1,186.0,39.0
2,2131.0,2133.0,211.0,40.0,0,2,186.0,40.0
3,2131.0,2134.0,234.0,40.0,0,2,90.0,40.0
4,2131.0,2135.0,161.0,41.0,0,2,142.0,41.0
5,2131.0,2136.0,186.0,43.0,0,2,246.0,44.0
6,2131.0,2138.0,211.0,45.0,0,2,114.0,46.0
7,2131.0,2139.0,233.0,48.0,0,2,162.0,48.0
8,2131.0,2141.0,162.0,49.0,0,2,237.0,49.0
9,2131.0,2142.0,236.0,51.0,0,2,43.0,51.0


## Create cruise dataset 

* with expanded rows if cruising more than 1 time interval

In [19]:
cruise_list = []
repeat_size = []
kept_list = []

# for cruise rows that are longer than 1 time interval
for row in reposition_df.itertuples(index=False):
    if (getattr(row, 'type') == 1) & (getattr(row, 'time_next') - getattr(row, 'time') > 1):
        cruise_list.append(tuple(row))
        repeat_size.append(getattr(row, 'time_next') - getattr(row, 'time'))
        kept_list.append(np.nan)
    else:
        kept_list.append(1)
reposition_df['kept'] = kept_list
reposition_df.dropna(subset=['kept'], inplace=True)
reposition_df.drop(columns=['kept'], inplace=True)

cruise_df = pd.DataFrame(np.repeat(np.array(cruise_list), repeat_size, axis=0))
cruise_df.columns = reposition_df.columns

## Cast type back to int
cruise_df = cruise_df.astype({'episode': 'float',
                              'sub_index': 'float',
                              'loc': 'float',
                              'time': 'float',
                              'loc_next': 'float',
                              'time_next': 'float',
                              'type': 'float'})
cruise_df = cruise_df.astype({'episode': 'Int64',
                              'sub_index': 'Int64',
                              'loc': 'Int64',
                              'time': 'Int64',
                              'loc_next': 'Int64',
                              'time_next': 'Int64',
                              'type': 'Int64'})

In [20]:
cruise_df.head(10)

,episode,sub_index,loc,time,reward,type,loc_next,time_next
0,3232,3257,238,77,0.0,1,238,79
1,3232,3257,238,77,0.0,1,238,79
2,3343,3356,138,62,0.0,1,138,64
3,3343,3356,138,62,0.0,1,138,64
4,3391,3392,138,37,0.0,1,138,39
5,3391,3392,138,37,0.0,1,138,39
6,3516,3536,132,69,0.0,1,132,75
7,3516,3536,132,69,0.0,1,132,75
8,3516,3536,132,69,0.0,1,132,75
9,3516,3536,132,69,0.0,1,132,75


In [21]:
## insert rows to dataframe if cruising takes more than 1 interval
def expand_cruise(selected_df):
    df = selected_df.copy()
    lower_time = min(df['time'])
    upper_time = max(df['time_next'])
    df['time'] = range(lower_time, upper_time)
    df['time_next'] = range(lower_time+1, upper_time+1)
    df['expanded_index'] = range(upper_time-lower_time) ##for sorting within expanded rows
    return df

### Expand and combine cruise df to reposition df

* resort the df using episode-sub_index key

In [22]:
cruise_df_2 = cruise_df.groupby(['episode', 'sub_index', 'loc', 'reward', 'type', 'loc_next']).apply(expand_cruise)
reposition_df = pd.concat([reposition_df, cruise_df_2], sort=True).sort_values(['episode', 'sub_index'])
reposition_df['expanded_index'] = reposition_df['expanded_index'].fillna(0)

In [23]:
reposition_df.head(10)

,episode,expanded_index,loc,loc_next,reward,sub_index,time,time_next,type
0,2131,0.0,166,238,0.0,2131,36.0,37.0,2
1,2131,0.0,186,186,0.0,2132,38.0,39.0,1
2,2131,0.0,211,186,0.0,2133,40.0,40.0,2
3,2131,0.0,234,90,0.0,2134,40.0,40.0,2
4,2131,0.0,161,142,0.0,2135,41.0,41.0,2
5,2131,0.0,186,246,0.0,2136,43.0,44.0,2
6,2131,0.0,211,114,0.0,2138,45.0,46.0,2
7,2131,0.0,233,162,0.0,2139,48.0,48.0,2
8,2131,0.0,162,237,0.0,2141,49.0,49.0,2
9,2131,0.0,236,43,0.0,2142,51.0,51.0,2


In [24]:
# transition_df = pd.concat([trip_df, reposition_df], sort=True)\
#                 .sort_values(['episode', 'sub_index', 'type']).reset_index()

# transition_df.reset_index(inplace=True)
## collaspe consecutive repo-delivery
# transition_df['mask'] = np.where((transition_df['type'] == 1) | (transition_df['type'] == 2), 
#                                  transition_df['index'], np.nan)
# transition_df['mask'] = transition_df.groupby('episode')['mask'].ffill()
# transition_df.drop(columns=['index', 'mask'], inplace=True)

In [25]:
# transition_df['next_action'] = transition_df.groupby('episode')['type'].shift(-1)
# transition_df['collapse'] = np.where(transition_df['next_action'] == 0, 1, 0)

In [26]:
# temp = transition_df.copy()
# temp['temp'] = temp.groupby('episode')['type'].shift(1)
# temp.loc[(temp['temp'] == temp['type'])]['type'].unique()[0] == 1 ##only action 1 can be repeated consecutively

In [27]:
sarsa_df = pd.concat([trip_df, reposition_df], sort=True)
sarsa_df['expanded_index'] = sarsa_df['expanded_index'].fillna(0)
sarsa_df.sort_values(['episode', 'sub_index', 'type', 'expanded_index'], inplace=True)

# sarsa_df.dropna(subset=['mask'], inplace=True)
sarsa_df['next_transition'] = sarsa_df.groupby('episode')['type'].shift(-1)

sarsa_df.head()

,episode,expanded_index,loc,loc_next,reward,sub_index,time,time_next,type,next_transition
0,2131,0.0,42,166,9.50,2131,36.0,36.0,0,2
0,2131,0.0,166,238,0.00,2131,36.0,37.0,2,0
1,2131,0.0,238,186,17.65,2132,37.0,38.0,0,1
1,2131,0.0,186,186,0.00,2132,38.0,39.0,1,0
2,2131,0.0,186,211,13.50,2133,39.0,40.0,0,2


In [28]:
ffill_cols = ['sub_index', 'loc', 'time']
for col_names in ffill_cols:
    sarsa_df[col_names] = np.where(sarsa_df['type'] == 0, ##empty delivery for ffill
                                  np.nan, sarsa_df[col_names])
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].ffill()
bfill_cols = ['loc_next', 'time_next']
for col_names in bfill_cols:
    sarsa_df[col_names] = np.where(((sarsa_df['type'] == 1) | (sarsa_df['type'] == 2)) &
                                        (sarsa_df['next_transition'] == 0), ## if repo and next action is delivery
                                  np.nan, sarsa_df[col_names]) ##empty repo for bfill
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].bfill()
sarsa_df.head()

,episode,expanded_index,loc,loc_next,reward,sub_index,time,time_next,type,next_transition
0,2131,0.0,NaN,166,9.50,NaN,NaN,36.0,0,2
0,2131,0.0,166,186,0.00,2131,36.0,38.0,2,0
1,2131,0.0,166,186,17.65,2131,36.0,38.0,0,1
1,2131,0.0,186,211,0.00,2132,38.0,40.0,1,0
2,2131,0.0,186,211,13.50,2132,38.0,40.0,0,2


## Combine reposition and trip dataset

## Convert into SARSA format

* collapse repo row with the next delivery row

In [29]:
sarsa_df = pd.concat([trip_df, reposition_df], sort=True)
sarsa_df['expanded_index'] = sarsa_df['expanded_index'].fillna(0)
sarsa_df.sort_values(['episode', 'sub_index', 'type', 'expanded_index'], inplace=True)

# sarsa_df.dropna(subset=['mask'], inplace=True)
sarsa_df['next_transition'] = sarsa_df.groupby('episode')['type'].shift(-1)
sarsa_df['action'] = sarsa_df['loc_next']

sarsa_df.head()

,episode,expanded_index,loc,loc_next,reward,sub_index,time,time_next,type,next_transition,action
0,2131,0.0,42,166,9.50,2131,36.0,36.0,0,2,166
0,2131,0.0,166,238,0.00,2131,36.0,37.0,2,0,238
1,2131,0.0,238,186,17.65,2132,37.0,38.0,0,1,186
1,2131,0.0,186,186,0.00,2132,38.0,39.0,1,0,186
2,2131,0.0,186,211,13.50,2133,39.0,40.0,0,2,211


In [30]:
ffill_cols = ['sub_index', 'expanded_index', 'loc', 'time', 'action']
for col_names in ffill_cols:
    sarsa_df[col_names] = np.where(sarsa_df['type'] == 0, ##empty delivery for ffill
                                  np.nan, sarsa_df[col_names])
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].ffill()
bfill_cols = ['loc_next', 'time_next']
for col_names in bfill_cols:
    sarsa_df[col_names] = np.where(((sarsa_df['type'] == 1) | (sarsa_df['type'] == 2)) &
                                        (sarsa_df['next_transition'] == 0), ## if repo and next action is delivery
                                  np.nan, sarsa_df[col_names]) ##empty repo for bfill
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].bfill()
sarsa_df.head()

,episode,expanded_index,loc,loc_next,reward,sub_index,time,time_next,type,next_transition,action
0,2131,NaN,NaN,166,9.50,NaN,NaN,36.0,0,2,NaN
0,2131,0.0,166,186,0.00,2131,36.0,38.0,2,0,238
1,2131,0.0,166,186,17.65,2131,36.0,38.0,0,1,238
1,2131,0.0,186,211,0.00,2132,38.0,40.0,1,0,186
2,2131,0.0,186,211,13.50,2132,38.0,40.0,0,2,186


In [31]:
sarsa_df = pd.concat([trip_df, reposition_df], sort=True)
sarsa_df['expanded_index'] = sarsa_df['expanded_index'].fillna(0)
sarsa_df.sort_values(['episode', 'sub_index', 'type', 'expanded_index'], inplace=True)

# sarsa_df.dropna(subset=['mask'], inplace=True)
sarsa_df['next_transition'] = sarsa_df.groupby('episode')['type'].shift(-1)
sarsa_df['action'] = sarsa_df['loc_next']

ffill_cols = ['sub_index', 'expanded_index', 'loc', 'time', 'action']
for col_names in ffill_cols:
    sarsa_df[col_names] = np.where(sarsa_df['type'] == 0, ##empty delivery for ffill
                                  np.nan, sarsa_df[col_names])
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].ffill()
bfill_cols = ['loc_next', 'time_next']
for col_names in bfill_cols:
    sarsa_df[col_names] = np.where(((sarsa_df['type'] == 1) | (sarsa_df['type'] == 2)) &
                                        (sarsa_df['next_transition'] == 0), ## if repo and next action is delivery
                                  np.nan, sarsa_df[col_names]) ##empty repo for bfill
    sarsa_df[col_names] = sarsa_df.groupby('episode')[col_names].bfill()
    
sarsa_df.dropna(subset=['next_transition'], inplace=True)
sarsa_df.drop(columns=['type', 'next_transition'], inplace=True)
sarsa_df = sarsa_df.astype('float64')
sarsa_df = sarsa_df.groupby(['episode', 'sub_index', 'expanded_index', 'loc', 
                             'loc_next', 'time', 'time_next', 'action'], as_index=False).agg('sum')
sarsa_df.sort_values(['episode', 'sub_index', 'expanded_index'], inplace=True)
sarsa_df = sarsa_df.astype({'episode': 'Int64',
                              'loc': 'Int64',
                              'time': 'Int64',
                              'loc_next': 'Int64',
                              'time_next': 'Int64'})
# # sarsa_df['action'] = sarsa_df['loc_next']
sarsa_df['action_next'] = sarsa_df.groupby('episode')['action'].shift(-1)
sarsa_df['state'] = [(loc, time) for loc, time in zip(sarsa_df['loc'], sarsa_df['time'])]
sarsa_df['state_next'] = [(loc, time) for loc, time in zip(sarsa_df['loc_next'], sarsa_df['time_next'])]
sarsa_df = sarsa_df[['episode', 'state', 'action', 'reward', 'state_next', 'action_next']]

In [32]:
sarsa_df.head()

,episode,state,action,reward,state_next,action_next
0,2131,"(166, 36)",238.0,17.65,"(186, 38)",186.0
1,2131,"(186, 38)",186.0,13.50,"(211, 40)",186.0
2,2131,"(211, 40)",186.0,8.45,"(234, 40)",90.0
3,2131,"(234, 40)",90.0,7.50,"(161, 41)",142.0
4,2131,"(161, 41)",142.0,27.10,"(186, 43)",246.0


In [33]:
import pickle
with open('../data/SARSA_eps.pickle', 'wb') as handle:
    pickle.dump(sarsa_df, handle)

## For interpretable read. Suffering data loss.
sarsa_df.to_csv('../data/SARSA_eps.csv', index=False)

In [ ]:
import pickle
with open(f'../data/SARSA_eps_{delta_t}m.pickle', 'wb') as handle:
    pickle.dump(sarsa_df, handle)

## For interpretable read. Suffering data loss.
sarsa_df.to_csv(f'../data/SARSA_eps_{delta_t}m.csv', index=False)

In [34]:
sarsa_df.shape

(3921625, 6)

In [35]:
## load pickle
# with open('../data/historical_for_SARSA.pickle', 'rb') as handle:
#     episode_data = pickle.load(handle)